In [1]:
#cleaning the data

In [33]:
import re
import pandas as pd
import numpy as np
import requests
import os
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from IPython.core.display import HTML
from datetime import date, datetime


In [34]:
senators_file= "1976-2018-senate.csv"

In [35]:
# new_senate=pd.read_csv(senators_file)

new_senate = pd.read_csv(senators_file, encoding= 'unicode_escape')


In [36]:
df= pd.DataFrame(new_senate)
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Sam Steiger,republican,False,total,321236,741210,False,20171011.0
1,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Wm. Mathews Feighan,independent,False,total,1565,741210,False,20171011.0
2,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Dennis DeConcini,democrat,False,total,400334,741210,False,20171011.0
3,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Allan Norwitz,libertarian,False,total,7310,741210,False,20171011.0
4,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Bob Field,independent,False,total,10765,741210,False,20171011.0


In [37]:
datayear = df[(df.year>=2014) & (df.year<=2018)]
datayear

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
2861,2014,Alabama,AL,1,63,41,US Senate,statewide,gen,False,NaN,NaN,True,total,22484,818090,False,20171011.0
2862,2014,Alabama,AL,1,63,41,US Senate,statewide,gen,False,Jeff Sessions,republican,False,total,795606,818090,False,20171011.0
2863,2014,Alaska,AK,2,94,81,US Senate,statewide,gen,False,NaN,NaN,True,total,1376,282400,False,20171011.0
2864,2014,Alaska,AK,2,94,81,US Senate,statewide,gen,False,Mark Begich,democrat,False,total,129431,282400,False,20171011.0
2865,2014,Alaska,AK,2,94,81,US Senate,statewide,gen,False,Mark S. Fish,libertarian,False,total,10512,282400,False,20171011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,55,35,25,US Senate,statewide,gen,False,NaN,NaN,True,total,42,2657841,False,20190110.0
3417,2018,Wyoming,WY,56,83,68,US Senate,statewide,gen,False,John Barrasso,republican,False,total,136210,203420,False,20190110.0
3418,2018,Wyoming,WY,56,83,68,US Senate,statewide,gen,False,Gary Trauner,democrat,False,total,61227,203420,False,20190110.0
3419,2018,Wyoming,WY,56,83,68,US Senate,statewide,gen,False,Joseph Porambo,libertarian,False,total,5658,203420,False,20190110.0


In [38]:
new_df=datayear[["year","state","state_po","candidate","party","candidatevotes","writein","totalvotes"]]
new_df

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,NaN,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,NaN,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,NaN,NaN,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [39]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 2861 to 3420
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            560 non-null    int64 
 1   state           560 non-null    object
 2   state_po        560 non-null    object
 3   candidate       468 non-null    object
 4   party           439 non-null    object
 5   candidatevotes  560 non-null    int64 
 6   writein         560 non-null    bool  
 7   totalvotes      560 non-null    int64 
dtypes: bool(1), int64(3), object(4)
memory usage: 35.5+ KB


In [40]:
new_df.columns

Index(['year', 'state', 'state_po', 'candidate', 'party', 'candidatevotes',
       'writein', 'totalvotes'],
      dtype='object')

In [41]:
new_df['candidate'] = new_df['candidate'].replace(np.NaN, "Write in")
new_df


/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,Write in,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,NaN,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [42]:
new_df

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,Write in,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,NaN,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [43]:
new_df['party'] = new_df['party'].replace(np.NaN, "No Party")
new_df


/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [44]:
new_df["party"]= new_df["party"].replace(["republican"], "Republican")
new_df

/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [45]:
new_df["party"]= new_df["party"].replace(["democrat"], "Democrat")
new_df

/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,Democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [46]:
new_df["party"]= new_df["party"].replace(["libertarian"], "Libertarian")
new_df

/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,Libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,Democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,Libertarian,5658,False,203420


In [47]:
new_df.shape

(560, 8)

In [48]:
# new_df.to_csv('votersinfo.csv', index = False)

In [49]:
new_df

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,Libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,Democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,Libertarian,5658,False,203420


In [50]:
votersInfo2014 = new_df[(new_df.year == 2014)]

In [51]:
#votersInfo2014

In [52]:
votersInfo2016 = new_df[(new_df.year == 2016)]
votersInfo2018 = new_df[(new_df.year == 2018)].reset_index()

In [53]:
chairs_file= "Chair_Members.csv"
matchingChairs = pd.read_csv(chairs_file, encoding= 'unicode_escape')

In [54]:
matchingChairs = matchingChairs[(matchingChairs.Committee == "Committee")].reset_index()
matchingChairs = matchingChairs.drop(['Committee', "C_Party", "C_Party_Name", "index"], axis = 1)
matchingChairs = matchingChairs.rename(columns= {'SubCommittee':'Committee'})


In [55]:
votersInfo2018 = votersInfo2018.drop(['year', 'writein', 'index'], axis = 1)
votersInfo2018 = votersInfo2018.rename(columns= {'state_po':'State_Abbrv', 'state':'State', 'candidate':'Candidate', 'party':'Party', 'candidatevotes':'Candidate_Votes', 'totalvotes':"Total_Votes"})
print(len(votersInfo2018))
votersInfo2018[:50]

152


,State,State_Abbrv,Candidate,Party,Candidate_Votes,Total_Votes
0,Arizona,AZ,Martha McSally,Republican,1135200,2384308
1,Arizona,AZ,Kyrsten Sinema,Democrat,1191100,2384308
2,Arizona,AZ,Angela Green,green,57442,2384308
3,Arizona,AZ,Write in,No Party,566,2384308
4,California,CA,Dianne Feinstein,Democrat,6019422,11113364
5,California,CA,Kevin De Leon,Democrat,5093942,11113364
6,Connecticut,CT,Christopher S Murphy,Democrat,787685,1386840
7,Connecticut,CT,Matthew Corey,Republican,545717,1386840
8,Connecticut,CT,Christopher S Murphy,working families,37894,1386840
9,Connecticut,CT,Richard Lion,Libertarian,8838,1386840


In [103]:
name = 'Angus S. "K"ing, Jr.'
noPeriod = ''
fixedList = []
counter = 0
for i in name:
    if (i == '.') & (counter < len(name)):
        counter += 1
        continue
    elif (i == "'") & (counter < len(name)):
        counter += 1
        continue
    elif (i == '"') & (counter < len(name)):
        counter += 1
        continue
    elif (i == ',') & (counter < len(name)):
        counter += 1
        continue
    elif counter <= len(name):
        noPeriod += i
        counter += 1
    else:
        counter += 1
fixedList.append(noPeriod)

In [104]:
noPeriod

'Angus S King Jr'

In [105]:
fixedList

['Angus S King Jr']

In [114]:
def noChars(stripList):
    cleanedList = []
    for longName in stripList:
        counter = 0
        noChars = ''
        for letter in longName:
            if (letter == '.') & (counter < len(longName)):
                counter += 1
                continue
            elif (letter == "'") & (counter < len(longName)):
                counter += 1
                continue
            elif (letter == '"') & (counter < len(longName)):
                counter += 1
                continue
            elif (letter == ',') & (counter < len(longName)):
                counter += 1
                continue
            elif counter <= len(name):
                noChars += letter
                counter += 1
            else:
                counter += 1
        cleanedList.append(noChars)
    return cleanedList


In [116]:
list2 = ['Angus S. "K"ing, Jr.', '"Bob" B. B,arker']
clean1 = noChars(list2)

In [161]:
def splitName(column):
    splitName = []
    for i in column:
        splitName.append(i.split())
    print(splitName)   
#     Namelist = []
#     noMiddle =[]
#     for i in column:
#         Namelist.append(i.split(' '))
#     #print(len(Namelist))
#     noMiddle = []
#     counter = 1
#     for i in Namelist:
#         for j in i:
#             if len(j) > 1:
#                 noMiddle.append(j)
#                 counter += 1
#             elif len(j) <= 2:
#                 noMiddle.append(j)
#                 counter += 1
#             else:
#                 counter += 1
#     print(noMiddle)
#     firstName = noMiddle[:len(noMiddle)+1:2]
#     middleI = noMiddle[1:len(noMiddle)+1:2]
#     add1 = len(firstName)
#     add2 = len(middleI)
#     lastName = noMiddle[2::3]
    print(len(firstName))
    print(len(middleI))
    return firstName, middleI, lastName

In [195]:
def splitName(column):
    splitName = []
    lastName = []
    for i in column:
        splitName.append(i.split())
    #print(len(splitName[0]))
    #print(splitName)
    firstName = []
    lastName = []
    middleI = []
    lastName = []
    suffix = []
    for i in splitName:
        #print(len(i))
        if len(i) == 2:
            firstName.append(i[0])
            lastName.append(i[1])
        if len(i) ==3:
            firstName.append(i[0])
            middleI.append(i[1])
            lastName.append(i[2])
        if len(i) == 4:
            firstName.append(i[0])
            middleI.append(i[1])
            suffix.append(i[-1])
            lastName.append(i[-2])
        #print(i)
    return firstName, middleI, lastName, suffix

In [198]:
splitNameList = splitName(clean1)
lastName = splitNameList[2]
lastName

['King', 'Barker']

In [31]:
splitVotersInfo2018 = splitName(votersInfo2018['Candidate'])
splitVotersInfo2018[0]

['Martha',
 'Kyrsten',
 'Angela',
 'Write',
 'Dianne',
 'Kevin',
 'Leon',
 'S',
 'Matthew',
 'Christopher',
 'Murphy',
 'Lion',
 'Russell',
 'in',
 'R',
 'Robert',
 'Arlett',
 'G',
 'Nadine',
 'Frost',
 'Scott',
 'Nelson',
 'in',
 'K.',
 'Ron',
 'Mike',
 'Joe',
 'Lucy',
 'Brenton',
 'in',
 'L.',
 'Angus',
 'King,',
 'Zak',
 'Others',
 'Campbell',
 'Cardin',
 'Vohra',
 'Simon',
 'in',
 'A.',
 'Geoff',
 'Shiva',
 'Others',
 'James',
 'Stabenow',
 'E.',
 'III',
 'Squier',
 'Howard',
 'Write',
 'Jim',
 'Amy',
 'Dennis',
 'Paula',
 'Overby',
 'in',
 'Housley',
 'Smith',
 'Wellington',
 'Trooien',
 'in',
 'Baria',
 'F.',
 'Danny',
 'Shawn',
 'Cindy',
 'Mike',
 'Josh',
 'Claire',
 'Japheth',
 'Jo',
 'Craig',
 'Write',
 'Jon',
 'Rick',
 'Matt',
 'Deb',
 'Jane',
 'Jim',
 'Write',
 'Jacky',
 'Dean',
 'None',
 'these',
 'Barry',
 'Tim',
 'Kamau',
 'Bakari',
 'Menendez',
 'Hugin',
 'R.',
 'Murray',
 'Natalie',
 'Rivera',
 'Flanagan',
 'Kimple',
 'Schroeder',
 'T',
 'Gary',
 'Johnson',
 'Rich',
 'E

In [27]:
findTVotes = votersInfo2018['Total_Votes']
findCVotes =

SyntaxError: invalid syntax (<ipython-input-27-f27a47d0ecec>, line 2)

In [ ]:
# candidate_Votes = []
# total_Votes = []
# for i in range(len(matchingChairs['Chair_Names'])):
#     if matchingChairs['Chair_Names'][i] in votersInfo2018['Candidate']:
#         candidate_Votes.append(votersInfo2018['Candidate_Votes'][i])
#         total_Votes.append(votersInfo2018['Total_Votes'][i])

In [ ]:
totalvotes = []
candidatevotes =[]
for i in range(len(votersInfo2018['Candidate'])):
    for j in range(len(matchingChairs['Chair_Names'])):
        if votersInfo2018['Candidate'][i].lower() == matchingChairs['Chair_Names'][j].lower():
            totalvotes.append(votersInfo2018['Total_Votes'][i])

In [ ]:
totalvotes

In [ ]:
print(len(matchingChairs))
matchingChairs

In [ ]:
# matchingChairs['Chair_Names'] 
# votersInfo2018['Candidate']

In [ ]:
# candidates = votersInfo2018['candidate']
# totalVotes = votersInfo2018['totalvotes']
# candidateVotes = votersInfo2018['candidatevotes']
# stateAbbr = votersInfo2018['state_po']
# chairs = matchingChairs['Chair_Names']

In [ ]:
# matchList = []
# for chair in chairs:
#     for candidate in candidates:
#         if chair.lower() in candidate.lower():
#             matchList.append(chair)